# Cloud Pak for Data Virtualization Demonstration
This notebook will connect to a CP4D cluster and demonstrate how SQL can be run against a variety of data sources without requiring any knowledge of how these data sources are accessed.

The first step is to load the Db2 extensions that allow us to run Db2 commands directly against CP4D.

In [ ]:
%run db2.ipynb

A connection to the database is required before we can run any SQL statements. Run the following statement to connect to the Cloud Pak for Data cluster. **Note:** If you need to change the connection information, edit the Connection document which is found in the table of contents.

In [ ]:
%run connection.ipynb

## Stock Symbol Table
### Get information about the stocks that are in the database
**System Z - VSAM**

In [ ]:
%sql -grid select * from DVDEMO.STOCK_SYMBOLS

## Top Buy/Sell By Customer (All Stocks)
**AWS - Db2, Azure - Db2 & Postgres, VMware - MongoDB**

In [ ]:
%%sql -grid
WITH
    TX_DAY(TX_DATE) AS (
      VALUES NEXT_DAY(DATE('2018-01-01') + INT(RAND()*350) DAYS, 'Thursday')
    ),
    PURCHASED(CUSTID,AMOUNT) AS (
      SELECT CUSTID, SUM(QUANTITY) AS AMOUNT
        FROM FOLDING.STOCK_TRANSACTIONS_DV ST, TX_DAY TX
      WHERE ST.TX_DATE = TX.TX_DATE
      GROUP BY CUSTID
    )
    SELECT C.LASTNAME, C.CITY, C.STATE, P.AMOUNT
      FROM PURCHASED P, TRADING.CUSTOMERS C
      WHERE C.CUSTID = P.CUSTID
    ORDER BY ABS(P.AMOUNT) DESC
    FETCH FIRST 9 ROWS ONLY 

## Stock History Table
### Get Price of a Stock over the Year
Set the Stock Symbol in the line below.

**CP4D - Db2, VMware - Informix**

In [ ]:
stock = 'INTC'

In [ ]:
%%sql -pl
SELECT WEEK(TX_DATE) AS WEEK, OPEN FROM FOLDING.STOCK_HISTORY
WHERE SYMBOL = :stock AND TX_DATE != '2017-12-01'
ORDER BY WEEK(TX_DATE) ASC

### Trend of Three Stocks
This chart shows three stock prices over the course of a year.

**CP4D - Db2, VMware - Informix**

In [ ]:
stocks = ['INTC','MSFT','AAPL']

In [ ]:
%%sql -pl
SELECT SYMBOL, WEEK(TX_DATE), OPEN FROM FOLDING.STOCK_HISTORY
WHERE SYMBOL IN (:stocks) AND TX_DATE != '2017-12-01'
ORDER BY WEEK(TX_DATE) ASC

### 30 Day Moving Average of a Stock
Enter the Stock Symbol below.

**CP4D - Db2, VMware -  Informix**

In [ ]:
stock = 'AAPL'

In [ ]:
sqlin = \
"""
SELECT WEEK(TX_DATE) AS WEEK, OPEN, 
     AVG(OPEN) OVER (
       ORDER BY TX_DATE
     ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING) AS MOVING_AVG
  FROM FOLDING.STOCK_HISTORY
     WHERE SYMBOL = :stock
  ORDER BY WEEK(TX_DATE)
"""
df = %sql {sqlin}
txdate= df['WEEK']
sales = df['OPEN']
avg = df['MOVING_AVG']

plt.xlabel("Day", fontsize=12);
plt.ylabel("Opening Price", fontsize=12);
plt.suptitle("Opening Price and Moving Average of " + stock, fontsize=20);
plt.plot(txdate, sales, 'r');
plt.plot(txdate, avg, 'b');
plt.show();

### Trading volume of INTC versus MSFT and AAPL in first week of November
**CP4D - Db2, VMware - Informix**

In [ ]:
stocks = ['INTC','MSFT','AAPL']

In [ ]:
%%sql -pb
SELECT SYMBOL, DAY(TX_DATE), VOLUME/1000000 FROM FOLDING.STOCK_HISTORY
WHERE SYMBOL IN (:stocks) AND WEEK(TX_DATE) =  45
ORDER BY DAY(TX_DATE) ASC

### Show Stocks that Represent at least 3% of the Total Purchases during Week 45
**CP4D - Db2, VMware -  Informix**

In [ ]:
%%sql -pie
WITH WEEK45(SYMBOL, PURCHASES) AS (
  SELECT SYMBOL, SUM(VOLUME * CLOSE) FROM FOLDING.STOCK_HISTORY
    WHERE WEEK(TX_DATE) =  45 AND SYMBOL <> 'DJIA'
  GROUP BY SYMBOL
),
ALL45(TOTAL) AS (
  SELECT SUM(PURCHASES) * .03 FROM WEEK45
)
SELECT SYMBOL, PURCHASES FROM WEEK45, ALL45
WHERE PURCHASES > TOTAL
ORDER BY SYMBOL, PURCHASES

## Stock Transaction Table
### Show Top Transaction Count by Customer
**AWS - Db2, Azure - Db2 & Postgres**

In [ ]:
%%sql
SELECT CUSTID, COUNT(*) FROM FOLDING.STOCK_TRANSACTIONS_DV
GROUP BY CUSTID
ORDER BY 2 DESC FETCH FIRST 10 ROWS ONLY

### Bought/Sold Amounts of Top 5 stocks 
**AWS - Db2, Azure - Db2, Postgres**

In [ ]:
%%sql
WITH BOUGHT(SYMBOL, AMOUNT) AS
  (
  SELECT SYMBOL, SUM(QUANTITY) FROM FOLDING.STOCK_TRANSACTIONS_DV
  WHERE QUANTITY > 0
  GROUP BY SYMBOL
  ),
SOLD(SYMBOL, AMOUNT) AS
  (
  SELECT SYMBOL, -SUM(QUANTITY) FROM FOLDING.STOCK_TRANSACTIONS_DV
  WHERE QUANTITY < 0
  GROUP BY SYMBOL
  )
SELECT B.SYMBOL, B.AMOUNT AS BOUGHT, S.AMOUNT AS SOLD
FROM BOUGHT B, SOLD S
WHERE B.SYMBOL = S.SYMBOL
ORDER BY B.AMOUNT DESC
FETCH FIRST 5 ROWS ONLY

## Customer Accounts
### Show Top 5 Customer Balance
**AWS - Db2, Azure - Db2, Postgres**

In [ ]:
%%sql
SELECT CUSTID, BALANCE FROM FOLDING.ACCOUNTS_DV
ORDER BY BALANCE DESC
FETCH FIRST 5 ROWS ONLY

### Show Bottom 5 Customer Balance
**AWS - Db2, Azure - Db2, Postgres**

In [ ]:
%%sql
SELECT CUSTID, BALANCE FROM FOLDING.ACCOUNTS_DV
ORDER BY BALANCE ASC
FETCH FIRST 5 ROWS ONLY

## Selecting Customer Information from MongoDB
The MongoDB database has customer information in a document format. In order to materialize the document data as relational tables, a total of four virtual tables are generated. The following query shows the tables that are generated for the Customer document collection.

In [ ]:
%sql LIST TABLES FOR SCHEMA MONGO_ONPREM

The tables are all connected through the CUSTOMERID field, which is based on the generated `_id` of the main CUSTOMER colllection. In order to reassemble these tables into a document, we must join them using this unique identifier. An example of the contents of the CUSTOMER_CONTACT table is shown below.

In [ ]:
%sql -grid SELECT * FROM MONGO_ONPREM.CUSTOMER_CONTACT FETCH FIRST 5 ROWS ONLY

A full document record is shown in the following SQL statement which joins all of the tables together.

In [ ]:
%%sql -grid
SELECT C.CUSTOMERID AS CUSTID, 
       CI.FIRSTNAME, CI.LASTNAME, CI.BIRTHDATE,
       CC.CITY, CC.ZIPCODE, CC.EMAIL, CC.PHONE, CC.STREET, CC.STATE,
       CP.CARD_TYPE, CP.CARD_NO
FROM MONGO_ONPREM.CUSTOMER C, MONGO_ONPREM.CUSTOMER_CONTACT CC, 
     MONGO_ONPREM.CUSTOMER_IDENTITY CI, MONGO_ONPREM.CUSTOMER_PAYMENT CP
WHERE  CC.CUSTOMER_ID = C."_ID" AND
       CI.CUSTOMER_ID = C."_ID" AND
       CP.CUSTOMER_ID = C."_ID"
FETCH FIRST 3 ROWS ONLY

## Querying All Virtualized Data
### What are the names of the customers in Ohio, who bought the most during the highest trading day of the year.
**AWS Db2, Azure Db2 & Postgres, VMware MongoDB, CP4D Db2Wh, VMware Informix**

In [ ]:
%%sql
WITH MAX_VOLUME(AMOUNT) AS (
  SELECT MAX(VOLUME) FROM FOLDING.STOCK_HISTORY
    WHERE SYMBOL = 'DJIA'
),
HIGHDATE(TX_DATE) AS (
  SELECT TX_DATE FROM FOLDING.STOCK_HISTORY, MAX_VOLUME M
    WHERE SYMBOL = 'DJIA' AND VOLUME = M.AMOUNT
),
CUSTOMERS_IN_OHIO(CUSTID) AS (
  SELECT C.CUSTID FROM TRADING.CUSTOMERS C 
    WHERE C.STATE = 'OH'
),
TOTAL_BUY(CUSTID,TOTAL) AS (
  SELECT C.CUSTID, SUM(SH.QUANTITY * SH.PRICE) 
    FROM CUSTOMERS_IN_OHIO C, FOLDING.STOCK_TRANSACTIONS_DV SH, HIGHDATE HD
  WHERE SH.CUSTID = C.CUSTID AND
        SH.TX_DATE = HD.TX_DATE AND 
        QUANTITY > 0 
  GROUP BY C.CUSTID
)
SELECT LASTNAME, T.TOTAL 
  FROM TRADING.CUSTOMERS C, TOTAL_BUY T
WHERE C.CUSTID = T.CUSTID
ORDER BY TOTAL DESC
FETCH FIRST 5 ROWS ONLY